# 3. FastAPI CRUD
In this notebook we evolve the FastAPI app from the previous lesson into a full CRUD service. You'll build endpoints to create, read, update, and delete tasks, then add a small search feature.

### What you'll practice
- Model request/response bodies with Pydantic
- Wire up CRUD endpoints using the appropriate HTTP verbs
- Use `TestClient` inside the notebook to exercise the API
- Extend the API with a small search enhancement

Run cells in order so the in-memory data stays consistent.


### Quick glossary
- **CRUD**: Create, Read, Update, Delete — the four basic operations an API often supports for a resource.
- **Endpoint**: A specific URL + HTTP method combination (e.g., `GET /tasks/`) that does one job.
- **Pydantic**: Library FastAPI uses to validate/serialize data using Python type hints.
- **BaseModel**: Pydantic base class; you subclass it to define the shape of request/response bodies.
- **FastAPI dependency injection**: FastAPI auto-creates and injects your model instances and path/query params from the HTTP request.
- **response_model**: FastAPI option that declares the type/shape returned to clients; it also filters fields.
- **status_code**: Explicit HTTP status returned (e.g., `201` for created, `204` for no content).
- **HTTPException**: Helper to return an HTTP error with status and message instead of raising a generic Python error.
- **TestClient**: Utility (from `fastapi.testclient`) that lets you call your API in tests without running a server.


In [1]:
# Imports
from fastapi import FastAPI, HTTPException
from fastapi.testclient import TestClient
from pydantic import BaseModel
from typing import List, Optional

## 3.1 Pydantic Models
Pydantic is a library for data validation using Python type hints. FastAPI uses it to define the "shape" of the data we expect to receive or send.

In [3]:
# Define a Pydantic model for a Task
class Task(BaseModel):                     # model describing the structure of a task object
    id: Optional[int] = None               # optional id field; will be assigned by the API
    title: str                             # required title field
    description: Optional[str] = None      # optional description field
    completed: bool = False                # default completion status

# Create the FastAPI app
app = FastAPI()                            # initializes the application instance

# Test client for calling the API in this notebook
client = TestClient(app)                   # allows making test requests without a server

# In-memory database
tasks_db = []                              # simple list storing task dictionaries during runtime


## 3.2 Create (POST)
We use the `POST` method to create new resources.
FastAPI will automatically validate the request body against the `Task` model.

In [4]:
# Create a new task
@app.post("/tasks/", response_model=Task, status_code=201)  # POST endpoint returning a Task with HTTP 201
def create_task(task: Task):                                # FastAPI validates the incoming JSON against Task
    task.id = len(tasks_db) + 1                             # assign a new ID based on current task count
    tasks_db.append(task)                                   # store the task in the in-memory database
    return task                                             # return the created task as JSON

# Test creating a task
new_task_data = {"title": "Learn FastAPI", "description": "It is cool"}  # request body sent to the API
response = client.post("/tasks/", json=new_task_data)                    # send POST request with JSON data
print(response.status_code)                                              # should be 201
print(response.json())                                                   # the created task with assigned id


201
{'id': 1, 'title': 'Learn FastAPI', 'description': 'It is cool', 'completed': False}


## 3.3 Read (GET)
We need endpoints to get all tasks and to get a specific task by ID.

In [5]:
# Read all tasks
@app.get("/tasks/", response_model=List[Task])     # returns a list of Task objects
def read_tasks():
    return tasks_db                                # send back all tasks stored in memory

# Read a specific task by ID
@app.get("/tasks/{task_id}", response_model=Task)  # "{task_id}" extracted from the path
def read_task(task_id: int):                       # FastAPI ensures task_id is an int
    for task in tasks_db:                          # search through stored tasks
        if task.id == task_id:                     # match by ID
            return task                            # return the matching task
    raise HTTPException(status_code=404,           # if no match, return a 404 error
                        detail="Task not found")

# Test reading all tasks
print("All tasks:", client.get("/tasks/").json())         # fetch all tasks

# Test reading one task
print("Task 1:", client.get("/tasks/1").json())           # fetch task with id=1

# Test reading non-existent task
print("Task 99:", client.get("/tasks/99").json())          # should return a 404 error message


All tasks: [{'id': 1, 'title': 'Learn FastAPI', 'description': 'It is cool', 'completed': False}]
Task 1: {'id': 1, 'title': 'Learn FastAPI', 'description': 'It is cool', 'completed': False}
Task 99: {'detail': 'Task not found'}


## 3.4 Update (PUT)
We use `PUT` to update an existing resource.

In [6]:
# Update an existing task
@app.put("/tasks/{task_id}", response_model=Task)         # PUT endpoint replaces the entire task
def update_task(task_id: int, updated_task: Task):        # validated Task model provided in request body
    for i, task in enumerate(tasks_db):                   # iterate to find the task by index
        if task.id == task_id:                            # match the ID in the path
            updated_task.id = task_id                     # keep the original ID
            tasks_db[i] = updated_task                    # replace the old task with the new version
            return updated_task                           # return the updated task
    raise HTTPException(status_code=404, detail="Task not found")  # if no match, respond with 404

# Test updating a task
update_data = {"title": "Learn FastAPI", "description": "It is SUPER cool", "completed": True}  # new task data
response = client.put("/tasks/1", json=update_data)                                             # send the update request
print("Updated Task:", response.json())                                                          # print updated task


Updated Task: {'id': 1, 'title': 'Learn FastAPI', 'description': 'It is SUPER cool', 'completed': True}


## 3.5 Delete (DELETE)
We use `DELETE` to remove a resource.

In [7]:
# Delete a task
@app.delete("/tasks/{task_id}", status_code=204)     # DELETE endpoint returning no content on success
def delete_task(task_id: int):                       # task_id taken from the path
    for i, task in enumerate(tasks_db):              # iterate with index to allow removal
        if task.id == task_id:                       # match the ID
            tasks_db.pop(i)                          # remove the task from the in-memory list
            return                                   # 204 response is sent automatically
    raise HTTPException(status_code=404,             # if not found, return 404
                        detail="Task not found")

# Test deleting a task
response = client.delete("/tasks/1")                 # attempt to delete task with id=1
print("Delete Status:", response.status_code)        # should show 204

# Verify it's gone
print("All tasks after delete:", client.get("/tasks/").json())  # remaining tasks


Delete Status: 204
All tasks after delete: []


## 3.6 Exercise: Search Feature
**Task**:
1. Modify the `read_tasks` endpoint (or create a new one like `/search/`) to accept a query parameter `keyword`.
2. If `keyword` is provided, return only tasks where the `keyword` is in the `title`.
3. If no `keyword` is provided, return all tasks.

### How to approach the search exercise
- Start with the existing `read_tasks` handler; reuse its return shape (`List[Task]`).
- Accept `keyword` as an optional query parameter and handle the `None` case first.
- Normalize text with `.lower()` for case-insensitive matches.
- Keep mutation out of this handler—just filter the current `tasks_db`.
- Add a couple of `client.get` calls to validate both filtered and unfiltered responses.


In [9]:
# Add sample tasks for search testing

sample_tasks = [                               # list of task dictionaries to populate the DB
    {"title": "Buy milk", "description": "Get 2% milk from store"},
    {"title": "Buy eggs", "description": "Get a dozen eggs"},
    {"title": "Walk the dog", "description": "30 minute walk in the park"},
    {"title": "Learn Python", "description": "Complete FastAPI tutorial"},
    {"title": "Learn FastAPI", "description": "Build a CRUD API"}
]

# Create sample tasks in the API (so the search feature has data to filter)
for task_data in sample_tasks:                     # iterate over each sample task dictionary
    client.post("/tasks/", json=task_data)         # create a new task in tasks_db by calling the POST /tasks/ endpoint


In [10]:
# Search endpoint to filter tasks by keyword

@app.get("/search/", response_model=List[Task])       # returns a list of Task objects
def search_tasks(keyword: Optional[str] = None):       # optional query parameter
    if not keyword:                                    # no keyword → return all tasks
        return tasks_db

    # return tasks whose titles contain the keyword (case-insensitive)
    return [task for task in tasks_db 
            if keyword.lower() in task.title.lower()]


In [ ]:
# Test: no keyword provided → return all tasks
print(client.get("/search/").json())                                # should list all tasks

# Test: keyword "Buy" → should match "Buy milk" and "Buy eggs"
print(client.get("/search/?keyword=Buy").json())

# Test: keyword "learn" → case-insensitive match
print(client.get("/search/?keyword=learn").json())

# Test: keyword with no matches
print(client.get("/search/?keyword=xyz").json())


[{'id': 1, 'title': 'Buy milk', 'description': 'Get 2% milk from store', 'completed': False}, {'id': 2, 'title': 'Buy eggs', 'description': 'Get a dozen eggs', 'completed': False}, {'id': 3, 'title': 'Walk the dog', 'description': '30 minute walk in the park', 'completed': False}, {'id': 4, 'title': 'Learn Python', 'description': 'Complete FastAPI tutorial', 'completed': False}, {'id': 5, 'title': 'Learn FastAPI', 'description': 'Build a CRUD API', 'completed': False}, {'id': 6, 'title': 'Buy milk', 'description': 'Get 2% milk from store', 'completed': False}, {'id': 7, 'title': 'Buy eggs', 'description': 'Get a dozen eggs', 'completed': False}, {'id': 8, 'title': 'Walk the dog', 'description': '30 minute walk in the park', 'completed': False}, {'id': 9, 'title': 'Learn Python', 'description': 'Complete FastAPI tutorial', 'completed': False}, {'id': 10, 'title': 'Learn FastAPI', 'description': 'Build a CRUD API', 'completed': False}]
[{'id': 1, 'title': 'Buy milk', 'description': 'Get 


### ✅ Implementation Summary

Key Features Implemented:
1. ✅ **Optional Query Parameter**: `keyword` is optional (defaults to `None`)
2. ✅ **Case-Insensitive Search**: Uses `.lower()` for both keyword and titles
3. ✅ **Returns All Tasks**: When no keyword is provided
4. ✅ **Filters by Title**: Searches for keyword within task titles
5. ✅ **Read-Only Operation**: No mutations to `tasks_db`
6. ✅ **Proper Response Model**: Returns `List[Task]`

The implementation includes **6 test cases**:
1. **No keyword** - Returns all 5 tasks
2. **Search "Buy"** - Finds "Buy milk" and "Buy eggs"
3. **Search "learn"** - Case-insensitive match for "Learn Python" and "Learn FastAPI"
5. **Search "xyz"** - Returns empty list (no matches)

This search pattern is commonly used for:
- Product searches in e-commerce
- Document filtering
- User searches
- Content discovery

### Solution

In [12]:
# Solution
# Note: In FastAPI, you can't easily "overwrite" an endpoint in the same app instance in a notebook cell 
# without redefining the app or using a different path. 
# For this exercise, let's create a specific search endpoint.

@app.get("/search/", response_model=List[Task])
def search_tasks(keyword: Optional[str] = None):
    if keyword:
        return [task for task in tasks_db if keyword.lower() in task.title.lower()]
    return tasks_db

# Add some dummy data to test
client.post("/tasks/", json={"title": "Buy milk"})
client.post("/tasks/", json={"title": "Buy eggs"})
client.post("/tasks/", json={"title": "Walk the dog"})

# Test search
print("Search 'Buy':", client.get("/search/?keyword=Buy").json())
print("Search 'dog':", client.get("/search/?keyword=dog").json())

Search 'Buy': [{'id': 1, 'title': 'Buy milk', 'description': 'Get 2% milk from store', 'completed': False}, {'id': 2, 'title': 'Buy eggs', 'description': 'Get a dozen eggs', 'completed': False}, {'id': 6, 'title': 'Buy milk', 'description': 'Get 2% milk from store', 'completed': False}, {'id': 7, 'title': 'Buy eggs', 'description': 'Get a dozen eggs', 'completed': False}, {'id': 11, 'title': 'Buy milk', 'description': None, 'completed': False}, {'id': 12, 'title': 'Buy eggs', 'description': None, 'completed': False}]
Search 'dog': [{'id': 3, 'title': 'Walk the dog', 'description': '30 minute walk in the park', 'completed': False}, {'id': 8, 'title': 'Walk the dog', 'description': '30 minute walk in the park', 'completed': False}, {'id': 13, 'title': 'Walk the dog', 'description': None, 'completed': False}]
